## **Advance Data Analysis**

In [ ]:
USE  DataWarehouse
SELECT Top 3 * FROM gold.fact_sales

## **Change over time analysis**

**Purpose:**

-     To track trends, growth, and changes in key metrics over time.
-     For time-series analysis and identifying seasonality.
-     To measure growth or decline over specific periods.

**SQL Functions Used:**

-  Date Functions: DATEPART(), DATETRUNC(), FORMAT()
-  Aggregate Functions: SUM(), COUNT(), AVG()

In [ ]:
SELECT 

    Year(order_date) AS Year,
    MONTH(order_date) AS Month,
    SUM(sales) AS total_sales,
    COUNT(DISTINCT customer_key) AS total_customers,
    SUM(quantity) AS total_quantity,
    COUNT(DISTINCT order_number) AS total_order

FROM 
    gold.fact_sales

WHERE order_date IS NOT NULL
GROUP BY YEAR(order_date), MONTH(order_date)
ORDER BY YEAR(order_date), MONTH(order_date)

In [ ]:
-- DATETRUNC()
SELECT
    DATETRUNC(month, order_date) AS order_date,
    SUM(sales) AS total_sales,
    COUNT(DISTINCT customer_key) AS total_customers,
    SUM(quantity) AS total_quantity
FROM gold.fact_sales
WHERE order_date IS NOT NULL
GROUP BY DATETRUNC(month, order_date)
ORDER BY DATETRUNC(month, order_date);

## **Cumulative Analysis**

**Purpose:**

-  To calculate running totals or moving averages for key metrics.
- To track performance over time cumulatively.
- Useful for growth analysis or identifying long-term trends.

**SQL Functions Used:**

- Window Functions: SUM() OVER(), AVG() OVER()

In [ ]:
-- Total sales for each month
-- Running toal of sales over month

SELECT
    order_date,
    Total_sales,
    Avg_price,
    SUM(Total_sales) OVER( ORDER BY order_date)  as running_total_sales,
    AVG(Avg_price) OVER( ORDER BY order_date)  as moving_average_price
FROM
(
        SELECT
            DATETRUNC(MONTH, order_date) as order_date,
            SUM(sales) as Total_sales,
            AVG(price) as Avg_price
        FROM
            gold.fact_sales
        WHERE
            DATETRUNC(MONTH, order_date) IS NOT NULL
        GROUP BY 
            DATETRUNC(MONTH, order_date)
) T


## **Performance Analysis**

**Purpose:**

- To measure the performance of products, customers, or regions over time.
- For benchmarking and identifying high-performing entities.
- To track yearly trends and growth.

**SQL Functions Used:**

- LAG(): Accesses data from previous rows.
- AVG() OVER(): Computes average values within partitions.
- CASE: Defines conditional logic for trend analysis.

In [ ]:
/* 
Analyze the yearly performance of product by comparing their sales to both the average sales performance of the product
and the previous year's sales
*/
-- year over year analysis just replacing year with month or any other partition will give you the corresponding analysis in time stamp

WITH yearly_product_sales as 
(
    SELECT 
        YEAR(f.order_date) as order_year,
        p.product_id,
        p.product_name,
        SUM(f.sales) as current_sales
    FROM 
        gold.dim_products as p 
    LEFT JOIN 
        gold.fact_sales as f
    on p.product_key = f.product_key 
    WHERE YEAR(f.order_date) IS NOT NULL
    GROUP BY 
        YEAR(f.order_date),
        p.product_id,
        p.product_name
)

SELECT 
    order_year,
    product_name,
    current_sales,
    AVG(current_sales) OVER (Partition by product_name) as avg_sales,
    current_sales - AVG(current_sales) OVER (Partition by product_name) as diff_avg,
    case

        when (current_sales - AVG(current_sales) OVER (Partition by product_name)) > 0 THEN 'Above average'
        when (current_sales - AVG(current_sales) OVER (Partition by product_name)) < 0 THEN 'Below average'
        ELSE 'Avg' 
 
    END as avg_change,
    LAG(current_sales) OVER(Partition by product_name ORDER BY order_year) as py_sales,
    (current_sales - LAG(current_sales) OVER(Partition by product_name ORDER BY order_year)) as py_diff,
    case

        when (current_sales - LAG(current_sales) OVER(Partition by product_name ORDER BY order_year)) > 0 THEN 'sales increase'
        when (current_sales - LAG(current_sales) OVER(Partition by product_name ORDER BY order_year)) < 0 THEN 'sales decrease'
        ELSE 'same' 
 
    END as py_change

FROM 
yearly_product_sales
ORDER BY  
product_name, order_year



## **Part to Whole analysis**

**Purpose:**

- <span style="color: var(--vscode-foreground);">To compare performance or metrics across dimensions or time periods.</span>
- <span style="color: var(--vscode-foreground);">To evaluate differences between categories.</span>
- <span style="color: var(--vscode-foreground);">Useful for A/B testing or regional comparisons.</span>

**SQL Functions Used:**

- SUM(), AVG(): Aggregates values for comparison.
- Window Functions: SUM() OVER() for total calculations.

In [ ]:
-- Which categories contribute the most to overall sales?

WITH category_sales as
(
    SELECT 
        p.category,
        SUM(f.sales) as sales_by_category
    FROM 
        gold.dim_products as p
    RIGHT JOIN
        gold.fact_sales as f
    on p.product_key = f.product_key
    GROUP BY p.category
) 

SELECT 
    category,
    sales_by_category,
    SUM(sales_by_category) OVER () as Total_sales,
    CONCAT(ROUND((CAST (sales_by_category AS FLOAT)/ SUM(sales_by_category) OVER ())*100, 2), ' %') as percentage_of_total  
FROM 
category_sales
ORDER BY sales_by_category DESC

## **Data segmentation**

**Purpose:**

- To group data into meaningful categories for targeted insigh
- <span style="color: var(--vscode-foreground);">For customer segmentation, product categorization, or regional analysis.</span>

**SQL Functions Used:**

- CASE: Defines custom segmentation logic.
- GROUP BY: Groups data into segments.

In [ ]:
-- segment products into cost ranges and count how many products fall into each segment

WITH product_segment AS
(
    SELECT
        product_key,
        product_name,
        cost,
        CASE 
            WHEN cost < 500 THEN  'Below 500'
            WHEN cost BETWEEN 500 and 1000 THEN '[500, 1000]'
            WHEN cost BETWEEN 1000 and 1500 THEN '[1000, 1500]'
            ELSE 'Above 1500'
        END cost_range
    FROM
        gold.dim_products 

)
SELECT
    cost_range,
    COUNT(product_key) as Total_products
FROM
product_segment
GROUP BY cost_range
ORDER BY Total_products DESC



In [ ]:
/*
Group customers into three segments based on their spending behaviour :

- VIP : at least 12 months of history and spending more than 5000
- Regular : at least 12 months of history but spending 5000 or less
- New : lifespan less than 12 months

Find the total number of customers by each group
*/

WITH Customers_lifetime_value AS
(
    SELECT
        c.customer_key,
        c.first_name,
        MIN(f.order_date) AS first_order_date,
        MAX(f.order_date) AS last_order_date,

        DATEDIFF(MONTH, MIN(f.order_date), MAX(f.order_date)) AS Lifespan,
        SUM(sales) as spendings
    FROM
        gold.dim_customers as c
        RIGHT JOIN
        gold.fact_sales as f
    on c.customer_key = f.customer_key
    GROUP BY 
    c.customer_key,
    c.first_name
)

SELECT 
customers_status,
COUNT(first_name) AS Total_customers
FROM
(
    SELECT
    first_name,
    Lifespan,
    spendings,
    CASE 
        WHEN Lifespan >= 12 and spendings > 5000 THEN 'VIP'
        WHEN Lifespan >= 12 and spendings <= 5000 THEN 'Regular'
        ELSE 'New'
    END as customers_status
    FROM
    Customers_lifetime_value
) T
GROUP BY customers_status
order by Total_customers





## **Customer Report**

<u><span style="color: rgb(0, 128, 0);"><br></span>**<span style="color: rgb(0, 128, 0);">Purpose:<br></span><span style="color: rgb(0, 128, 0);">- This reports consolidates key customers metirics and behaviours</span>**</u>

<span style="color: #008000;"><u><b>Highlights:</b></u></span>

<span style="color: #008000;"><u><b>1. Gather essential fields such as names, ages, and transaction details.</b></u></span>

<span style="color: #008000;"><u><b>2. Segments customers into categories (VIP, Regular, New) and age groups.</b></u></span>

<span style="color: #008000;"><u><b>3. Aggregates customer-level metrics.</b></u></span>

> > <span style="color: #008000;"><u><b>- Total orders</b></u></span>
> 
> > <span style="color: #008000;"><u><b>- Total sales</b></u></span>
> 
> > <span style="color: #008000;"><u><b>- Total quantity purchased</b></u></span>
> 
> > <span style="color: #008000;"><u><b>- Total products</b></u></span>
> 
> > <span style="color: #008000;"><u><b>- Lifespans (in months)</b></u></span>

<span style="color: #008000;"><u><b>4. Calculates valuable KPIs</b></u></span>

> > <span style="color: #008000;"><u><b>- recency (months since last order)</b></u></span>
> 
> > <span style="color: #008000;"><u><b>- average order value</b></u></span>
> 
> > <span style="color: #008000;"><u><b>- average monthly spend</b></u></span>

<span style="color: #008000;"><u><b>=========================================================================================</b></u></span>

In [ ]:
CREATE VIEW gold.report_customers AS
-- Base query CTE
WITH base_query AS
(
    SELECT 
        f.order_number,
        f.product_key,
        f.order_date,
        f.sales,
        f.quantity,
        c.customer_key,
        c.customer_number,
        CONCAT(c.first_name, ' ', c.last_name) AS Name,
        DATEDIFF(YEAR, c.birth_date, GETDATE()) AS Age
    FROM gold.fact_sales AS f
    LEFT JOIN gold.dim_customers AS c
        ON f.customer_key = c.customer_key
    WHERE f.order_date IS NOT NULL
),

-- Aggregation CTE
cust_aggregation AS
(
    SELECT 
        customer_key,
        customer_number,
        Age,
        COUNT(DISTINCT product_key) AS Total_products,
        SUM(sales) AS Total_sales,
        SUM(quantity) AS Total_quantity,
        COUNT(DISTINCT order_number) AS Total_orders,
        MIN(order_date) AS First_order,
        MAX(order_date) AS Last_order,
        DATEDIFF(MONTH, MIN(order_date), MAX(order_date)) AS Lifespan
    FROM base_query
    GROUP BY 
        customer_key,
        customer_number,
        Age
)

-- Final result
SELECT 
    customer_key,
    customer_number,
    Age,
    CASE 
        WHEN Age < 20 THEN 'Below 20'
        WHEN Age BETWEEN  20 AND 29 THEN '[20, 29]'
        WHEN Age BETWEEN 30 and 39 THEN  '[30, 39]'
        WHEN Age BETWEEN 40 and 49 THEN  '[40, 49]'
        ELSE '50 and above'
    END as Age_group,
    CASE 
        WHEN Lifespan >= 12 and Total_sales > 5000 THEN 'VIP'
        WHEN Lifespan >= 12 and Total_sales <= 5000 THEN 'Regular'
        ELSE 'New'
    END as customers_status,
    Last_order,
    DATEDIFF(MONTH, Last_order, GETDATE()) AS Recency,
    Total_products,
    Total_sales,
    Total_quantity,
    Total_orders,
    First_order,
    Lifespan,
    -- average order value
    CASE 
        WHEN Total_orders = 0 THEN 0
        ELSE
        Total_sales / Total_orders
    END as average_order_value,

    -- on avg monthly customers expenditure
    CASE 
        WHEN Lifespan = 0 THEN Total_sales
        ELSE 
        Total_sales / Lifespan
    END as 'Monthy expenditure' 
FROM cust_aggregation;



In [ ]:
select * from gold.report_customers

## **Product Report**

<span style="color: #008000;"><b><u>Purpose</u></b></span>

<span style="color: #008000;"><b><u>- This report consolidates key products metrics and behaviors.</u></b></span>

<span style="color: #008000;"><b><u>Highlights:</u></b></span>

<span style="color: #008000;"><b><u>1. Gathers essentials fields such as product name, category, subcategory, and cost.</u></b></span>

<span style="color: #008000;"><b><u>2. Segments products by revenue to identify High-Performance, Mid-Range, or Low-Performance.</u></b></span>

<span style="color: #008000;"><b><u>3. Aggregates product level metrics :</u></b></span>

> > <span style="color: #008000;"><b><u>- Total orders</u></b></span>
> 
> > <span style="color: #008000;"><b><u>- Total sales</u></b></span>
> 
> > <span style="color: #008000;"><b><u>- Total quantity sold</u></b></span>
> 
> > <span style="color: #008000;"><b><u>- Total customers (unique)</u></b></span>
> 
> > <span style="color: #008000;"><b><u>- lifespan in (Months)</u></b></span>

<span style="color: #008000;"><b><u>4. Calculate valuable KPIs:</u></b></span>

> > <span style="color: #008000;"><b><u>- recency (Months since last sale)</u></b></span>
> 
> > <span style="color: #008000;"><b><u>- average order revenue</u></b></span>
> 
> > <span style="color: #008000;"><b><u>- average monthly revenue</u></b></span>

In [ ]:
/*
==========================================================================================================
Product report
==========================================================================================================
Purpose 
 - This report consolidates key products metrics and behaviors.

 Highlights:
     1. Gathers essentials fields such as product name, category, subcategory, and cost.
     2. Segments products by revenue to identify High-Performance, Mid-Range, or Low-Performance.
     3. Aggregates product level metrics :
        - Total orders
        - Total sales
        - Total quantity sold
        - Total customers (unique)
        - lifespan in (Months)
    4. Calculate valuable KPIs:
        - recency (Months since last sale)
        - average order revenue
        - average monthly revenue

==========================================================================================================   
*/

In [ ]:
CREATE VIEW gold.report_products AS
-- Base query
WITH base_query AS
(
    SELECT
        p.product_key,
        p.product_name,
        p.category,
        p.subcategory,
        p.cost,
        f.order_date,
        f.order_number,
        f.sales,
        f.quantity,
        f.customer_key
    FROM
        gold.fact_sales as f
    LEFT JOIN
        gold.dim_products as p
    on 
        f.product_key = p.product_key
    WHERE order_date IS NOT NULL

),

-- Aggregation query

product_aggregations AS
(
SELECT
        product_key,
        product_name,
        category,
        subcategory, 
        COUNT(DISTINCT order_number) as Total_orders,
        SUM(sales) as Total_sales,
        SUM(quantity) as Total_quantity, 
        COUNT(DISTINCT customer_key) as Total_customers,
        MIN(order_date) AS First_order,
        MAX(order_date) AS Last_order,
        DATEDIFF(MONTH,MIN(order_date) , MAX(order_date)) as Lifespan
        
FROM
base_query
GROUP BY 
        product_key,
        product_name,
        category,
        subcategory
)

-- Final query for putting alltogether
SELECT 
        product_key,
        product_name,
        category,
        subcategory, 
        Total_orders,
        Total_sales,

        CASE 
            WHEN Total_sales <= 10000 THEN 'Low performance'
            WHEN Total_sales BETWEEN 10000 AND 50000 THEN 'Mid range'
            ELSE 'High performance'
        END AS Product_performance,
        
        Total_quantity, 
        Total_customers,
        First_order,
        Last_order,


-- Recency dim calculation
        DATEDIFF(MONTH, Last_order, GETDATE()) AS Recency,
        CASE
            WHEN Total_orders = 0 THEN 0
            ELSE
            Total_sales/Total_orders
        END As Average_order_revenue,

-- Average monthly revenue
        CASE
            WHEN Lifespan = 0 THEN Total_sales
            ELSE
            Total_sales/Lifespan
        END As Average_monthly_revenue,
        Lifespan



FROM
product_aggregations

In [ ]:
SELECT * FROM gold.report_products